# Republic vs Democratic USA politican parties Tweets

### Importing the needed modues or libraries

In [24]:
import pandas as pd
import numpy as np

import re
import string

import nltk
from nltk import *
from nltk.corpus import stopwords
import nltk as nlp
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# OS file system
from os import listdir
from os.path import isfile, join

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Read the Data

In [2]:
path = './data/tweets_2019/'
allFilesNames = [f for f in listdir(path) if isfile(join(path, f))]

FileNotFoundError: [Errno 2] No such file or directory: './data1/tweets_2019/'

In [3]:
df_twtr = pd.read_csv('./data1/H115_tw_2019.csv')
# Set the names as index of the table
df_twtr.set_index('WikiPageName',inplace=True)
# Fix Gene Green twitter tag
df_twtr.at['Gene_Green','tw_name'] = 'RepGeneGreen'
df_twtr.head(10)

FileNotFoundError: [Errno 2] File ./data1/H115_tw_2019.csv does not exist: './data1/H115_tw_2019.csv'

## Data Cleaning

In [4]:
# Stop words are loaded
stop_words = stopwords.words('english')

# Tweet path is specified and csv-file is loaded
names = df_twtr.index.to_numpy()
handles = [w.lower() for w in df_twtr.tw_name.to_numpy()]

# Pattern for regex
pattern = r'''(?x)
            (?:[A-Z]\.)+
            | [a-zA-Z]+(?:-[a-zA-Z]+)*'''

# Empty lists for tokens
dem_tokens = []
rep_tokens = []

for name in allFilesNames:
    if(name != '.DS_Store'):
        # Files are opened
        f = open(path+name, 'r', encoding='ISO-8859-1').read()
        # Removes newline character
        f = f.replace('\n', ' ')
        # Removes commonly used ampersand
        f = f.replace('&amp;', ' ')
        # Removes ' from contractions such as I'm and don't
        f = f.replace('’', '')
        # Lowercases the string
        f = f.lower()
        f = f.translate(str.maketrans('', '', string.punctuation))
        # Remove https-links from the string
        f = re.sub(r'(\bhttps\w+\b)', '', f)
        # Tokenizing the string with the given pattern

        tokens = regexp_tokenize(f, pattern)

        # If a word in the token list is not a stopword, not a twitterhandle of another member and longer than
        # two characters, it is added to the correct party-list
        if df_twtr.loc[name]['Party'] == 'Democratic':
            [dem_tokens.append(w) for w in tokens if w not in handles and len(
                w) > 2 and w not in stop_words]
        else:
            [rep_tokens.append(w) for w in tokens if w not in handles and len(
                w) > 2 and w not in stop_words]

NameError: name 'df_twtr' is not defined

### Compute the ferquency of each word

In [5]:
# Frequency distributions of the parties
dem_freqdist = FreqDist(dem_tokens)
rep_freqdist = FreqDist(rep_tokens)

# Vocabulary is saved as is a total frequency distribution
vocab = set(dem_tokens+rep_tokens)
freqdist = FreqDist(dem_tokens+rep_tokens)

# Most common terms for each party:
print("Most common Democratic terms:")
[print('\t- '+w) for w in list(dict(dem_freqdist.most_common(5)).keys())]
print("Most common Republican terms:")
[print('\t- '+w) for w in list(dict(rep_freqdist.most_common(5)).keys())]

NameError: name 'dem_tokens' is not defined

### Plot the most used words

In [6]:
import matplotlib.pyplot as plt
plt.subplots(figsize=(10,5))
dem_freqdist.plot(30,title="Democrat Tweets")
plt.subplots(figsize=(10,5))
rep_freqdist.plot(30,title="Republican Tweets")

NameError: name 'dem_freqdist' is not defined

### WordCloud of the most used words

In [7]:
unique_string_democratic = (" ").join(list(dict(dem_freqdist.most_common(100000)).keys()))
unique_string_republican = (" ").join(list(dict(rep_freqdist.most_common(100000)).keys()))
# Wordclouds are created. Collocations is set to false to avoid multiple instances of the same word.
# Colours are set to represent the parties.
dem_wordcloud = WordCloud(max_font_size=100, max_words=100, collocations=False,
                          background_color='white', colormap=plt.cm.Blues, width=800, height=800).generate(unique_string_democratic)
rep_wordcloud = WordCloud(max_font_size=100, max_words=100, collocations=False,
                          background_color='white', colormap=plt.cm.Reds, width=800, height=800).generate(unique_string_republican)

NameError: name 'dem_freqdist' is not defined

In [8]:
# Plotting of the wordclouds
fig = plt.figure(figsize=(10, 8))

# Democratic wordcloud
plt.subplot(1, 2, 1)
plt.imshow(dem_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Wordcloud for Democrats', fontsize=15, y=1.04)

# Republican wordcloud
plt.subplot(1, 2, 2)
plt.imshow(rep_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Wordcloud for Republicans', fontsize=15, y=1.04)

plt.savefig("./img/WordCloud100"+".pdf", bbox_inches='tight')
plt.show()

NameError: name 'dem_wordcloud' is not defined

OBSERVATION: we see that both parties' most used words are fairly common words in the field of politics. We therefore calculate the TFIDF for each party.

### WordCloud with TFIDF

In [9]:
# The IDF calculations can take 1 of 2 values: either a word is in both documents in which case idf = log(2/2) = 0, or
# The word is in only one document so that idf = log(2/1) = log(2)

# IDF value is pre-calculated for easy retrieval
c = np.log(2)
# Dataframes are created for both parties containg their respective vocabularies
df_dem = pd.DataFrame(set(dem_tokens), columns=['word'])
df_rep = pd.DataFrame(set(rep_tokens), columns=['word'])
# The dataframes are concatenated using 'outer'. The resulting word-column will be the total
# vocaulary. The second column will be called either 'both' (if word is in both vocabs), 'left_only'
# (word is only in Rep. vocab) or 'right_only' (word is only in Dem. vocab)
df_IDF = pd.merge(df_rep, df_dem, how='outer', indicator=True)
# Column labels
df_IDF.columns = ['word', 'party']
# Column called IDF is created which is a copy of the merger (i.e., 'both', 'left_only', 'right_only')
df_IDF['IDF'] = df_IDF['party']
# It is checked whether the value is 'both'. This returns a boolean (False = 0, True = 1).
# By multiplying these booleans with the calculated IDF, c, all instances where the word is only in
# one party vocab, the value will be c. Where it is 'both', the value will be 0.
df_IDF['IDF'] = (df_IDF['IDF'] != 'both')*c
# Index is changed
df_IDF.set_index('word', inplace=True)
# Frequency distribution of the words
df_freq = pd.DataFrame({'freq': dict(freqdist)})
# Dataframe is created containing the IDF and frequency distribution
df_TFIDF = df_IDF.join(df_freq)
# Values are multiplied resulting in the TFIDF
df_TFIDF['TFIDF'] = df_TFIDF['IDF']*df_TFIDF['freq']

NameError: name 'dem_tokens' is not defined

In [10]:
# An empty string is created. A word is added as many times as its TFIDF-value (rounded to nearest integer)
# This is done for both parties
dem_t = ''.join([(index+' ')*int(row['TFIDF']) for index,
                 row in df_TFIDF[df_TFIDF['party'] == 'right_only'].iterrows()])
rep_t = ''.join([(index+' ')*int(row['TFIDF']) for index,
                 row in df_TFIDF[df_TFIDF['party'] == 'left_only'].iterrows()])

NameError: name 'df_TFIDF' is not defined

In [11]:
# Wordclouds are created. Collocations is set to false to avoid multiple instances of the same word.
# Colours are set to represent the parties.
dem_wordcloud = WordCloud(max_font_size=100, max_words=100, collocations=False,
                          background_color='white', colormap=plt.cm.Blues, width=800, height=800).generate(dem_t)
rep_wordcloud = WordCloud(max_font_size=100, max_words=100, collocations=False,
                          background_color='white', colormap=plt.cm.Reds, width=800, height=800).generate(rep_t)

NameError: name 'dem_t' is not defined

In [12]:
# Plotting of the wordclouds
fig = plt.figure(figsize=(10, 8))

# Democratic wordcloud
plt.subplot(1, 2, 1)
plt.imshow(dem_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Wordcloud for Democrats', fontsize=15, y=1.04)

# Republican wordcloud
plt.subplot(1, 2, 2)
plt.imshow(rep_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Wordcloud for Republicans', fontsize=15, y=1.04)
plt.savefig("./img/WordCloudTFIDF"+".pdf", bbox_inches='tight')

plt.show()

NameError: name 'dem_wordcloud' is not defined

OBSERVATION: the wordclouds are very interesting. We see that the Democrats apparently have a larger focus on the climate and health care where the Republicans focus on anti-abortion and tax-cuts. These topics are all more interesting and polarising than the previous results, where we saw that both parties often referred to different political personalities, committees and other political jargon. We see a clear indicator, that the phrases for both parties are political slogans, such as endgunviolence and bornalive. Twitter therefore gives us a valuable insight into the key-issus and focal points of each party.

## Building the Classifier

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import SpatialDropout1D

from nltk import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

np.random.seed(1)

Using TensorFlow backend.


### Data Cleaning

In [14]:
top_words = 10000  # We'll keep these many distinct tokens and drop less frequent ones

# Stop words are loaded
stop_words = stopwords.words('english')

# Tweet path is specified and csv-file is loaded
names = df_twtr.index.to_numpy()
handles = [w.lower() for w in df_twtr.tw_name.to_numpy()]

# Pattern for regex
pattern = r'''(?x)
            (?:[A-Z]\.)+
            | [a-zA-Z]+(?:-[a-zA-Z]+)*'''

# Empty lists for tokens
all_party = []
all_handle = []
all_tokens = []

for name in allFilesNames:
    if(name != '.DS_Store'):
        # Files are opened
        f = open(path+name, 'r', encoding='utf-8').read()
        # Split the file into tweets
        f = f.split('\n')
        # Removes newline character
        for tweet in f:
            tweet = tweet.replace('\n', ' ')
            # Removes commonly used ampersand
            tweet = tweet.replace('&amp;', ' ')
            # Removes ' from contractions such as I'm and don't
            tweet = tweet.replace('’', '')
            # Lowercases the string
            tweet = tweet.lower()
            tweet = tweet.translate(str.maketrans('', '', string.punctuation))
            # Remove https-links from the string
            tweet = re.sub(r'(\bhttps\w+\b)', '', tweet)
            # Tokenizing the string with the given pattern
            tokens = regexp_tokenize(tweet, pattern)

            # If a word in the token list is not a stopword, not a twitterhandle of another member and longer than
            # two characters, it is added to the correct party-list
            list_token = []
            if df_twtr.loc[name]['Party'] == 'Democratic':
                [list_token.append(w) for w in tokens if w not in handles and len(
                    w) > 2 and w not in stop_words]
                if list_token != []:
                    all_party.append('Democrat')
                    all_handle.append(name)
                    all_tokens.append(list_token)
            else:
                [list_token.append(w) for w in tokens if w not in handles and len(
                    w) > 2 and w not in stop_words]
                if list_token != []:
                    all_party.append('Republican')
                    all_handle.append(name)
                    all_tokens.append(list_token)

NameError: name 'df_twtr' is not defined

In [15]:
dict = {'Party': all_party,
        'Handle': all_handle,
        'Tweet': all_tokens}
 
# creating a dataframe from dictionary
all_tweets = pd.DataFrame(dict)
all_tweets = shuffle(all_tweets)

NameError: name 'all_party' is not defined

In [16]:
all_tweets.head(10)

NameError: name 'all_tweets' is not defined

In [17]:
# Get the word counts
fdist = FreqDist(word for tweet in all_tweets.Tweet for word in tweet)
# Get top top_words terms, in order of frequency
terms = [term for term, count in fdist.most_common(top_words)]
# Replace all tokens with their rank (or 0 if not in the top top_words)
all_tweets.Tweet = all_tweets.Tweet.apply(lambda tweet: [terms.index(term) if term in terms else 0
                                                         for term in tweet])

all_tweets.head(10)

NameError: name 'all_tweets' is not defined

In [18]:
# Get an 80/20 train/test split of Twitter handles, stratified on Party
tweeters = all_tweets.iloc[:,:2].drop_duplicates()
handles_train, handles_test = train_test_split(tweeters.Handle, stratify=tweeters.Party, test_size=0.2, random_state=0)

# extract train & test sets from the all_tweets df
train = all_tweets[all_tweets.Handle.isin(handles_train)].reset_index().drop('index', axis=1)
test = all_tweets[all_tweets.Handle.isin(handles_test)].reset_index().drop('index', axis=1)

NameError: name 'all_tweets' is not defined

In [19]:
# The purpose of this cell is just to format the data to match the format of keras.datasets.imdb
X_train = np.array(train.Tweet)
y_train = np.array((train.Party == 'Democrat').astype(int))
X_test = np.array(test.Tweet)
y_test = np.array((test.Party == 'Democrat').astype(int))

NameError: name 'train' is not defined

In [20]:
# truncate and pad input sequences
max_review_length = 48 # The longest tweet is 56 tokens, and only a handfull are longer than 48
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

NameError: name 'X_train' is not defined

In [21]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(SpatialDropout1D(rate=0.1))

model.add(Conv1D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(SpatialDropout1D(rate=0.1))

model.add(Conv1D(filters=128, kernel_size=1, padding='same', activation='relu'))
model.add(Conv1D(filters=128, kernel_size=1, padding='same', activation='relu'))
model.add(Conv1D(filters=128, kernel_size=1, padding='same', activation='relu'))
model.add(SpatialDropout1D(rate=0.1))

model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

NameError: name 'Dropout' is not defined

In [22]:
preds = np.round(model.predict(X_test).ravel())
print('Accuracy: {}'.format(str(round(accuracy_score(y_test, preds), 4))))

NameError: name 'X_test' is not defined

In [23]:
accuracy_df = test.drop(['Tweet'], axis=1)
accuracy_df['preds'] = preds
accuracy_df['Party'] = y_test
correct = 0
total = 0

print("Congresspersons whose tweets were mostly mis-classified:")
for name in accuracy_df.Handle.unique():
    sub_df = accuracy_df[accuracy_df.Handle == name].reset_index()
    sub_preds = sub_df.preds.value_counts()

    if sub_preds.index[0] == sub_df.Party[0]:
        correct += 1
    total += 1

    if sub_preds[sub_df.Party[0]]/len(sub_df) < 0.5:
        print("{} ({}) classified with accuracy: {}".format(name, sub_df.Party[0],
                                                                      str(round(sub_preds[sub_df.Party[0]]/len(sub_df), 4))))

print()
print("Accuracy of the model on a per-Congressperson-basis was: {}".format(str(round(correct/total, 4))))

NameError: name 'test' is not defined